In [11]:
import pandas as pd
import re
import sys
sys.path.append("../")
from clean import TextCleaner

from transformers import BertPreTrainedModel, BertConfig, BertModel
from transformers import BertTokenizer
import torch
import math
import numpy as np
from patent_utils import *
import gc
from encode import BERTEncoder
from BERTSimilarity import DocumentBert, BERTSimilarityTrainer
from torchviz import make_dot

In [2]:
# Load the dataset into a pandas dataframe.
df = pd.read_csv("patent-tsd-matches-01192021.csv")

# Get only train data
patent_documents, patent_test = split_train_test(df.patentClaim1.values)
tsd_documents, tsd_test = split_train_test(df.TSDSection.values)
labels, labels_test = split_train_test(df.match.values)

In [3]:
bertEncoder = BERTEncoder(patent_documents, tsd_documents, labels, PatentCleaner())
patent_representations, tsd_representations, correct_output, tsd_dict = bertEncoder.tokenize_train_data()
patent_representations.shape, tsd_representations.shape, correct_output.shape

(torch.Size([2172, 3, 3, 512]),
 torch.Size([2172, 1, 3, 512]),
 torch.Size([2172]))

In [4]:
bertSimilarityTrainer = BERTSimilarityTrainer(patent_documents, tsd_documents, labels, threshold=0.8)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing DocumentBert: ['bert.encoder.layer.9.intermediate.dense.weight', 'bert.encoder.layer.8.attention.self.query.weight', 'bert.encoder.layer.2.attention.output.dense.bias', 'bert.encoder.layer.7.attention.self.query.bias', 'bert.encoder.layer.1.attention.output.dense.bias', 'bert.encoder.layer.9.attention.self.query.weight', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.1.intermediate.dense.bias', 'bert.encoder.layer.6.attention.output.LayerNorm.weight', 'bert.encoder.layer.8.output.dense.bias', 'bert.encoder.layer.6.attention.self.key.bias', 'bert.embeddings.token_type_embeddings.weight', 'cls.predictions.transform.dense.weight', 'bert.encoder.layer.4.attention.output.LayerNorm.bias', 'bert.encoder.layer.7.attention.self.value.bias', 'bert.encoder.layer.4.intermediate.dense.bias', 'bert.encoder.layer.1.attention.self.value.bias', 'bert.encoder.layer.5.attention.output.dense.bias

Some weights of DocumentBert were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert_tsd.encoder.layer.10.attention.self.key.bias', 'bert_patent.encoder.layer.4.attention.self.value.bias', 'lstm_patent.bias_hh_l0', 'bert_patent.encoder.layer.2.attention.self.key.bias', 'bert_tsd.encoder.layer.3.attention.self.key.weight', 'bert_tsd.encoder.layer.4.output.LayerNorm.bias', 'bert_tsd.encoder.layer.0.output.dense.bias', 'bert_tsd.encoder.layer.0.attention.self.value.weight', 'bert_tsd.encoder.layer.6.output.dense.bias', 'bert_patent.encoder.layer.0.attention.output.dense.bias', 'bert_patent.encoder.layer.4.intermediate.dense.weight', 'bert_tsd.encoder.layer.0.attention.self.query.bias', 'bert_patent.encoder.layer.5.attention.self.query.bias', 'bert_patent.encoder.layer.0.output.dense.weight', 'bert_patent.encoder.layer.10.attention.output.LayerNorm.bias', 'bert_tsd.encoder.layer.8.intermediate.dense.weight', 'bert_tsd.encoder.layer.5.output.Lay

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
bertSimilarityTrainer.train()

In [5]:
model = bertSimilarityTrainer.model

NameError: name 'bertSimilarityTrainer' is not defined

In [8]:
patent_claims = 'low and midrange bit rate transmission path the header specified by TCP UDP IP or other protocols is larger in size disadvantageously causing overhead associated with communication. For example to transmit 10-byte data under UDP/IP the transmitting side has to add a 25'
patent_claims_list = [patent_claims] * len(tsd_dict)
bertEncoder_predict = BERTEncoder(patent_documents=patent_claims_list, patentCleaner=PatentCleaner())

In [10]:
model = torch.load('models/04-09-2021.23.03.07_BERTsimilaritymodel.pth')

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:


# model.predict(patent_claims, bertEncoder_predict, tsd_representations)

patent_representations, output = bertEncoder_predict.encode_patents(patent_claims_list)

patent_representations = patent_representations.to('cpu')
tsd_representations = tsd_representations.to('cpu')

# patent_representations = patent_representations.to('cpu')
# tsd_representations = tsd_representations.to('cpu')
model.to('cpu')

similarity_scores = []
batch_size = 4
gc.collect()
torch.cuda.empty_cache()

for i in range(0, patent_representations.shape[0], batch_size):

    batch_patent_tensors = patent_representations[i:i + batch_size].to(device='cpu')
    batch_tsd_tensors = tsd_representations[i:i + batch_size].to(device='cpu')
    batch_predictions = model(batch_patent_tensors,
                              batch_tsd_tensors, 
                              device='cpu'
                             )
    similarity_scores.extend(batch_predictions)
    
section_scores_dict = {}
phrases_scores_dict = {}

for doc_index in tsd_dict.keys():
    section_scores_dict[doc_index] = predictions[tsd_dict[doc_index]].mean()
    phrases_scores_dict[doc_index] = predictions[tsd_dict[doc_index]].mean()